<a href="https://colab.research.google.com/github/juanazorzolo/TP-CV-ZORZOLO/blob/main/TP_CV_ZORZOLO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **TRABAJO PRÁCTICO FINAL CV 2025 - SISTEMA DE DETECCIÓN Y CLASIFICACIÓN DE RAZAS DE PERROS**

**AUTOR: Juana Zorzolo Rubio (Z-1217/3)**

Objetivo General:

Desarrollar un pipeline completo de visión por computadora para la identificación de razas de perros en imágenes. El proyecto abarca desde la creación de un sistema de búsqueda por similitud hasta la implementación de un sistema de detección y clasificación en imágenes complejas, incluyendo el entrenamiento y la optimización de modelos de Deep Learning.


# Etapa 1: Buscador de Imágenes por Similitud

## Creación de la Base de Datos Vectorial

In [38]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (2).json


{'kaggle (2).json': b'{"username":"juanazorzolo","key":"16e8ca4c7eebb08f98f32ced7754996d"}'}

In [39]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [40]:
!kaggle datasets download -d gpiosenka/70-dog-breedsimage-data-set
!unzip -q 70-dog-breedsimage-data-set.zip -d /content/

Dataset URL: https://www.kaggle.com/datasets/gpiosenka/70-dog-breedsimage-data-set
License(s): CC0-1.0
70-dog-breedsimage-data-set.zip: Skipping, found more recently modified local copy (use --force to force download)
replace /content/dogs.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace /content/test/Afghan/01.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace /content/test/Afghan/02.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace /content/test/Afghan/03.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [41]:
# PREPARACIÓN DEL ENTORNO

!pip install -q gradio faiss-cpu torchvision
import os
import numpy as np
from PIL import Image
from tqdm import tqdm
import torch
import torchvision.transforms as transforms
from torchvision.models import resnet50
import faiss

In [42]:
# Mostrar las carpetas que existen en /content
print("Contenido de /content:")
for nombre in os.listdir("/content"):
    print("-", nombre)

Contenido de /content:
- .config
- test
- .gradio
- train
- dogs.csv
- kaggle.json
- valid
- 70-dog-breedsimage-data-set.zip
- temp.jpg
- kaggle (2).json
- kaggle (1).json
- sample_data


In [43]:
# CARGAR EL MODELO PREENTRENADO (ResNet 50) (sin la capa final de clasificación)
model = resnet50(pretrained=True)
model = torch.nn.Sequential(*(list(model.children())[:-1]))  # Quitar la última capa (fc)
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)


In [44]:
# PREPROCESAMIENTO DE IMÁGENES

# Transformaciones requeridas por ResNet
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],  # Imagenet
        std=[0.229, 0.224, 0.225]
    )
])

In [45]:
# FUNCIÓN PARA EXTRAER LOS EMBEDDINGS

def extract_embedding(img_path):
    image = Image.open(img_path).convert('RGB')
    img_tensor = transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        embedding = model(img_tensor).squeeze().cpu().numpy()
    return embedding

In [46]:
# PROCESAR EL DATASET Y CONSTRUIR LOS VECTORES

# Cargar todas las imágenes y extraer embeddings
image_paths = []
embeddings = []

dataset_paths = ["/content/train", "/content/valid"] #saque , "/content/test" para usarlo después (VER ESTO)
image_paths = []
embeddings = []

for dataset_path in dataset_paths:
    for root, dirs, files in os.walk(dataset_path):
        for file in files:
            if file.lower().endswith((".jpg", ".jpeg", ".png")):
                path = os.path.join(root, file)
                try:
                    emb = extract_embedding(path)
                    image_paths.append(path)
                    embeddings.append(emb)
                except Exception as e:
                    print(f"Error al procesar {path}: {e}")

embeddings = np.array(embeddings).astype('float32')
print(f"Total imágenes procesadas: {len(embeddings)}")

Total imágenes procesadas: 8646


In [47]:
# INDEXAR EN BDD VECTORIAL (FAISS)
# Crear el índice FAISS
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

## Desarrollo de la Aplicación en Gradio

In [24]:
# FUNCIÓN DE BUSQUEDA EN FAISS
def buscar_similares(img_path, k=10):
  'toma una imagen, extrae su embedding, busca en FAISS, devuelve las 10 rutas más similares.'
  query_emb = extract_embedding(img_path).astype('float32').reshape(1, -1)
  distances, indices = index.search(query_emb, k)
  resultados = [image_paths[i] for i in indices[0]]
  return resultados

In [26]:
# INTERFAZ EN GRADIO
import gradio as gr
def interfaz_gradio(imagen):
    # Guardar la imagen temporalmente
    temp_path = "/content/temp.jpg"
    imagen.save(temp_path)

    # Buscar similares
    similares = buscar_similares(temp_path)

    # Cargar las imágenes similares
    resultados = [Image.open(p) for p in similares]

    return resultados

In [48]:
# Lanzar interfaz
gr.Interface(
    fn=interfaz_gradio,
    inputs=gr.Image(type="pil"),
    outputs=[gr.Image(type="pil", label=f"Similar #{i+1}") for i in range(10)],
    title="Buscador de Razas de Perros por Similitud",
    description="Subí una imagen de un perro y te mostramos las 10 más similares del dataset."
).launch(debug=True)

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://cc0542b2a2f1ce36e9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.11/dist-packages/gradio/blocks.py:1895: UserWarning: A function (interfaz_gradio) returned too many output values (needed: 10, returned: 12). Ignoring extra values.
    Output components:
        [image, image, image, image, image, image, image, image, image, image]
    Output values returned:
        [<PIL.Image.Image image mode=RGB size=612x512 at 0x7D7D02AABE90>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x7D7D021C9590>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x7D7D021CAA50>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x7D7D021E0150>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x7D7D021E1490>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x7D7D035BECD0>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x7D7D03594B90>, <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x7D7D021CFE90>, <PIL.JpegImagePlugin.JpegIm

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://cc0542b2a2f1ce36e9.gradio.live


## Clasificación Basada en Similitud y Métrica de Evaluación

In [28]:
# PARA CLASIFICACIÓN POR VOTO MAYORITARIO, ES NECESARIO EXTRAER LA RAZA REAL DESDE EL PATH DE CADA IMAGEN
# como en el dataset las carpetas se llaman como la raza

def extraer_raza(path):
    return os.path.basename(os.path.dirname(path))

In [29]:
# en interfaz_gradio, se agrega el voto mayoritario sobre las razas de las 10 imágenes recuperadas
from collections import Counter

def interfaz_gradio(imagen):
    temp_path = "/content/temp.jpg"
    imagen.save(temp_path)

    similares = buscar_similares(temp_path)
    resultados = [Image.open(p) for p in similares]

    # Voto mayoritario
    razas = [extraer_raza(p) for p in similares]
    raza_predicha = Counter(razas).most_common(1)[0][0]

    return [imagen] + resultados + [f"Raza predicha: {raza_predicha}"]

In [49]:
# cambio la interfaz para que también muestre la imagen subida y el texto

gr.Interface(
    fn=interfaz_gradio,
    inputs=gr.Image(type="pil"),
    outputs=[gr.Image(type="pil", label="Imagen de entrada")] +
            [gr.Image(type="pil", label=f"Similar #{i+1}") for i in range(10)] +
            [gr.Textbox(label="Raza Predicha")],
    title="Buscador de Razas de Perros por Similitud",
    description="Subí una imagen de un perro y te mostramos las 10 más similares del dataset y la raza más probable."
).launch(debug=True)

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://cc8ec2dd0fe7abbe1b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://cc8ec2dd0fe7abbe1b.gradio.live


In [50]:
#  Cálculo de NDCG@10

def extraer_raza(path):
    return os.path.basename(os.path.dirname(path))

def dcg(relevancias):
    return sum(rel / np.log2(idx + 2) for idx, rel in enumerate(relevancias))

def ndcg(relevancias):
    ideal = sorted(relevancias, reverse=True)
    return dcg(relevancias) / (dcg(ideal) + 1e-8)

def evaluar_ndcg(test_images, k=10):
    scores = []

    for img_path in tqdm(test_images):
        true_raza = extraer_raza(img_path)
        similares = buscar_similares(img_path, k=k)
        razas_similares = [extraer_raza(p) for p in similares]

        relevancias = [1 if raza == true_raza else 0 for raza in razas_similares]
        score = ndcg(relevancias)
        scores.append(score)

    ndcg_promedio = np.mean(scores)
    print(f"NDCG@{k} promedio: {ndcg_promedio:.4f}")

In [51]:
# Ejemplo: usar 5 imágenes de prueba por raza
from collections import defaultdict

test_set = []
razas_vistas = defaultdict(int)

for root, dirs, files in os.walk("/content/test"):
    for file in files:
        if file.lower().endswith((".jpg", ".jpeg", ".png")):
            raza = os.path.basename(root)
            if razas_vistas[raza] < 5:
                test_set.append(os.path.join(root, file))
                razas_vistas[raza] += 1

print(f"Imágenes de prueba: {len(test_set)}")

Imágenes de prueba: 350


In [52]:
evaluar_ndcg(test_set, k=10)

100%|██████████| 350/350 [00:05<00:00, 59.97it/s]

NDCG@10 promedio: 0.9733


# Etapa 2: Entrenamiento y Comparación de Modelos de Clasificación

## Entrenamiento de Modelos

### Modelo A (Transfer Learning)

### Modelo B (Opcional, recomendado)

## Integración y Selección en la Aplicación

# Etapa 3: Pipeline de Detección y Clasificación en Escenas Complejas

## Detección de Objetos

## Creación del Pipeline Completo

# Etapa 4: Evaluación, Optimización y Herramientas de Anotación

## Evaluación del Pipeline

## Optimización de Modelos (Elegir una)

## Script de Anotación Automática